In [2]:
import os
# ggf muss Pfad angepasst werden
os.chdir("{}/..".format(os.getcwd()))
os.getcwd()


'/home/lukas/Projects/dmc2019'

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, linear_model, metrics
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, cross_validate
#from scripts.utils import own_scorer, calc_scores

## Data

In [4]:
df_train = pd.read_csv("data/extended_train.csv", sep="|")
sum_frauds, sum_non_frauds  = len(df_train[df_train.fraud == 1]), len(df_train[df_train.fraud == 0])
df_y = df_train.fraud
df_X = df_train.drop(['fraud'], axis=1)

train_x, val_x, train_y, val_y = train_test_split(df_X, df_y, test_size=0.2)

print("Shape of train data: ", train_x.shape, "Shape of val data: ", val_x.shape)
df_train.head()

Shape of train data:  (1503, 10) Shape of val data:  (376, 10)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScannedLineItems
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0,29.0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0,14.0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0,13.0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0,29.0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0,27.0


## Data with new Features

In [71]:
df = df_train
df['totalScannedLineItems'] = df['scannedLineItemsPerSecond'] * df['totalScanTimeInSeconds']
df['avgTimePerScan'] = 1/ df['scannedLineItemsPerSecond']
df['avgValuePerScan'] = df['avgTimePerScan'] * df['valuePerSecond']
df['withoutRegisPerPosition'] = df['scansWithoutRegistration'] / df['totalScannedLineItems'] #equivalent to lineItemVoidsPerPosition?
df['quantiModPerPosition'] = df['quantityModifications'] / df['totalScannedLineItems']
df['lineItemVoidsPerTotal'] = df['lineItemVoids'] / df['grandTotal']
df['withoutRegisPerTotal'] = df['scansWithoutRegistration'] / df['grandTotal']
df['quantiModPerTotal'] = df['quantityModifications'] / df['grandTotal']
df['lineItemVoidsPerTime'] = df['lineItemVoids'] / df['totalScanTimeInSeconds']
df['withoutRegisPerTime'] = df['scansWithoutRegistration'] / df['totalScanTimeInSeconds']
df['quantiModPerTime'] = df['quantityModifications'] / df['totalScanTimeInSeconds']
df['valuePerScannedLineItem'] = df['valuePerSecond'] / df['scannedLineItemsPerSecond']
df1_y = df.fraud
df1_X = df.drop(['fraud'], axis=1)

In [49]:
print("Frauds: ", sum_frauds, "Non Frauds: ", sum_non_frauds)

Frauds:  104 Non Frauds:  1775


### Scoring Methods

In [6]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172)}

In [ ]:
params = {
    "max_depth": randint(2, 6),  # default 3
    "n_estimators": randint(300, 450),  # default 100
    "eta" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "lambda" : uniform(0.1,2),
    "scale_pos_weight" : [1, sum_non_frauds/sum_frauds],
    "max_bin" : randint(200, 300)
}
default_xgb = xgb.XGBClassifier(booster="gbtree",tree_method='gpu_hist', disable_default_eval_metric=1,objective='binary:logistic',eval_metric='aucpr', n_jobs=-1, verbosity=2)
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=10000,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='FBeta')
search.fit(df_X, df_y)
results = search.cv_results_

Fitting 3 folds for each of 10000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 21.8min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 25.4min
[Parallel(n_jobs=-1)]: Done 11226 tasks      

In [29]:
def own_scorer_normalized(estimator, X_val, ground_truth):
    prediction = estimator.predict(X_val)
    confusion_matrix = metrics.confusion_matrix(ground_truth, prediction)
    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))
    return dmc_score/len(ground_truth)

# DMC Score for usage as scorer
def own_scorer(estimator, X_val, ground_truth):
    prediction = estimator.predict(X_val)
    confusion_matrix = metrics.confusion_matrix(ground_truth, prediction)
    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))
    return dmc_score

scorings = {"DMC" : own_scorer, "DMC_Norm" : own_scorer_normalized}
xgbo = search.best_estimator_
res = cross_validate(xgbo, df_X, df_y, scoring=scorings, cv=5, n_jobs=-1)

In [30]:
res

/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_DMC'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_DMC_Norm'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([2.02021098, 2.02655697, 2.02314067, 2.00755906, 1.96474791]),
 'score_time': array([0.00790453, 0.00854206, 0.00603199, 0.00687265, 0.008425  ]),
 'test_DMC': array([10, 35, 95, 40, 30]),
 'test_DMC_Norm': array([0.02659574, 0.09308511, 0.25265957, 0.10638298, 0.08      ]),
 'train_DMC': array([415, 415, 415, 415, 420]),
 'train_DMC_Norm': array([0.27611444, 0.27611444, 0.27611444, 0.27611444, 0.27925532])}

In [31]:
results

{'mean_fit_time': array([1.57679733, 1.77067812, 1.93467339, ..., 1.62852995, 1.4885145 ,
        0.98766621]),
 'mean_score_time': array([0.01343004, 0.01779056, 0.01591404, ..., 0.00731587, 0.00730467,
        0.01186585]),
 'mean_test_AUC': array([0.99654747, 0.9956154 , 0.96381256, ..., 0.99763167, 0.99835683,
        0.96390425]),
 'mean_test_FBeta': array([0.85629233, 0.71191035, 0.75662265, ..., 0.87965747, 0.84673023,
        0.        ]),
 'mean_train_AUC': array([0.99965954, 0.99831877, 0.9767591 , ..., 0.99998775, 1.        ,
        0.97533494]),
 'mean_train_FBeta': array([0.97123925, 0.74744778, 0.79580985, ..., 0.98987908, 0.9498165 ,
        0.        ]),
 'param_eta': masked_array(data=[0.3621780831931537, 0.7063233020424545,
                    0.22727747704497042, ..., 0.7151330437877423,
                    0.4665844622959594, 0.69272689145796],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 '

In [32]:
search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eta=0.1769182150877735, eval_metric='aucpr',
       gamma=1.8285912697052542, lambda=0.4149772770711012,
       learning_rate=0.1, max_bin=254, max_delta_step=7.2556696256684035,
       max_depth=3, min_child_weight=1.0317712458399741, missing=None,
       n_estimators=445, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, tree_method='gpu_hist', verbosity=2)